## Dataset

In [1]:
import time
import random as rd
import csv

In [2]:
def crea_dataset():
    column_names=['server_id', 'application_id', 'start_timestamp']
    server_list = [x for x in range(1,11)]
    application_list = ['app'+ str(x)  for x in range(1,101)]
    start_time_list=[int(time.time())+x for x in range(1,1000)]
    data=[]
    data.append(column_names)
    for i in range(1000):
        raw=[i,rd.choice(server_list),rd.choice(application_list), rd.choice(start_time_list)]
        data.append(raw)
    with open('dataset.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(data)
    return data    

In [3]:
crea_dataset();

## Analysis

In [4]:
import numpy as np
import pandas as pd

In [5]:
df= pd.DataFrame.from_csv('dataset.csv')

In [6]:
df.head()

,server_id,application_id,start_timestamp
0,2,app62,1508340395
1,6,app44,1508340370
2,8,app88,1508340785
3,1,app77,1508340947
4,6,app59,1508339968


In [7]:
df.columns

Index(['server_id', 'application_id', 'start_timestamp'], dtype='object')

In [8]:
df=df.sort_values(by=['start_timestamp'], ascending=[True])

In [9]:
df = df.reset_index(drop=True)
df[:5]

,server_id,application_id,start_timestamp
0,4,app72,1508339967
1,6,app59,1508339968
2,3,app12,1508339969
3,7,app5,1508339970
4,4,app27,1508339971


In [11]:
df.groupby(df['server_id']).get_group(2).describe()

,server_id,start_timestamp
count,94.0,9.400000e+01
mean,2.0,1.508340e+09
std,0.0,2.829008e+02
min,2.0,1.508340e+09
25%,2.0,1.508340e+09
50%,2.0,1.508340e+09
75%,2.0,1.508341e+09
max,2.0,1.508341e+09


In [12]:
p=pd.Series()
for i in range(1,11):    
    s=df.groupby(['server_id'])['start_timestamp'].get_group(i).diff().shift(-1)
    p=p.append(s)
p.sort_index()
df['app_time']=p

In [13]:
df['app_time'].describe()

count    990.000000
mean       9.878788
std        9.889657
min        0.000000
25%        3.000000
50%        7.000000
75%       14.000000
max       96.000000
Name: app_time, dtype: float64

In [14]:
df['app_time'].isnull().sum()

10

In [20]:
df_filtered=df[(df.app_time > 45 ) & (df.app_time < 300)]
df_filtered

,server_id,application_id,start_timestamp,app_time
20,1,app11,1508339987,96.0
25,3,app96,1508339999,46.0
334,4,app23,1508340306,54.0
498,5,app98,1508340466,51.0
770,6,app43,1508340735,47.0
824,6,app89,1508340784,55.0
825,8,app88,1508340785,52.0
855,3,app78,1508340821,53.0
910,4,app90,1508340881,53.0


In [10]:
df['application_id'].groupby(df['server_id']).describe()

,count,unique,top,freq
server_id,,,,
1,100,58,app89,4
2,94,62,app15,3
3,105,62,app95,4
4,102,67,app88,4
5,99,60,app4,5
6,93,67,app59,4
7,92,59,app59,4
8,91,62,app37,4
9,123,69,app68,5
